# Collecting Tweet Data from Twitter API

### Authenticating Access

In [1]:
import tweepy
import csv
import sys
import datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

consumer_key = "xzUQDPt5xWi2vnFHkEPnNbox1"
consumer_secret = "HfDQSgcX0h2A6xHZhfXOYyc6sXkNEnY52RqvL3kPvGIjywQLPV"

access_token = "1126929832548257792-EJn3G8R0q7iDOX75SUAflrMT6OKT6F"
access_token_secret = "UhL3ErCAPxd73LLrqujOgPTQHjWfqQGKHxwyFolPnt3ou"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

### Connecting to SQLite to Build Database

In [2]:
import sqlite3

conn = sqlite3.connect('capstone2.db')
cursor = conn.cursor()

In [3]:
def create_table(name, col_details):
    cursor.execute('CREATE TABLE IF NOT EXISTS ' + name + '(' + col_details + ')')
    
def data_entry(name, values):
    cursor.execute('INSERT INTO ' + name + ' VALUES' + '(' + values + ')')
    
import re
pat = r'RT'
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
pat3 = r'([^0-9A-Za-z \t])'
combined_pat = r'|'.join((pat, pat1, pat2, pat3))
def tweet_cleaner(text):
    stripped = re.sub(combined_pat, '', text)
    return stripped

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()
def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    for key in list(score.keys()):
        new_key = key + '_sent'
        score[key + '_sent'] = score.pop(key)
    return(score)

I've made 2 key functions here that would help with the sentiment analysis.

The function *clean_tweet* was created using the re (regular expression) package that removes special characters and hyperlinks from the tweet so that it can be analyzed more accurately.

The sentiment analyzer used is called VADER Sentiment Analysis. VADER (Valence Aware Dictionary and sEntiment Reasoner) is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media.

Created Necessary Tables in Database

In [78]:
tables = ['netflix_tweets', 'disney_tweets', 'amazon_tweets', 'google_tweets']
for table in tables:
    create_table(table, 
             'created_at DATETIME, tweet VARCHAR(200), follower_count INT, neg_sent FLOAT, neu_sent FLOAT, pos_sent FLOAT, compound_sent FLOAT')

### Setting up Twitter API Search

Wrote query to select appropriate tweets

In [4]:
s = ' OR '
netflix = '@Netflix OR $NFLX OR Netflix'
disney = ['@Disney', '@ESPN', '@ABCnetwork', '@Pixar', '@Marvel', '$DIS']
disney = s.join(disney)
amazon = ['@Amazon', '@PrimeVideo', '@awscloud', '@TwitchPrime', '@Alexa', '@WholeFoods', '$AMZN']
amazon = s.join(amazon)
google = ['@Google', '@Android', '@Waymo', '$GOOGL']
google = s.join(google)

In [9]:
cursor.close()

### Inserting the tweets from Twitter API into local database

In [ ]:
for tweet in tweepy.Cursor(api.search, q= amazon, lang="en", since_id="2019-06-15",  until = "2019-06-16", tweet_mode='extended').items():
    follower_count = tweet.user.followers_count
    data = {'created_at': tweet.created_at.date(), 'tweet': tweet.full_text, 'follower_count': follower_count}
    data.update(sentiment_analyzer_scores(tweet_cleaner(tweet.full_text)))
    columns = ', '.join(data.keys())
    placeholders = ':'+', :'.join(data.keys())
    query = 'INSERT INTO amazon_tweets (%s) VALUES (%s)' % (columns, placeholders)
    cursor.execute(query, data)
    conn.commit()

# Collecting Daily Stock Data

In [61]:
from alpha_vantage.timeseries import TimeSeries
ts = TimeSeries(key='D0D213LZDE18R8MB',  output_format='pandas')

data, meta_data = ts.get_daily('GOOGL')
data.reset_index(level=0, inplace=True)

# Filter for only the dates needed
google_stock = data[data.date > '2019-05-14'].reset_index(drop=True)
google_stock.columns = ['date', 'open', 'high', 'low', 'close', 'volume']
google_stock

,date,open,high,low,close,volume
0,2019-05-15,1122.55,1178.30,1121.40,1170.80,2965117.0
1,2019-05-16,1171.84,1194.16,1168.45,1184.50,1765388.0
2,2019-05-17,1175.83,1186.29,1166.42,1168.78,1268050.0
3,2019-05-20,1153.00,1153.00,1138.13,1144.66,1530126.0
4,2019-05-21,1154.48,1158.00,1143.31,1154.44,1028248.0
5,2019-05-22,1151.25,1163.78,1151.00,1155.85,941279.0
6,2019-05-23,1146.07,1150.05,1133.16,1145.34,1260657.0
7,2019-05-24,1152.00,1154.36,1136.71,1138.61,927651.0
8,2019-05-28,1141.48,1156.49,1138.67,1139.56,1047554.0
9,2019-05-29,1132.70,1135.00,1111.95,1119.94,1811510.0


In [7]:
from alpha_vantage.timeseries import TimeSeries
ts = TimeSeries(key='D0D213LZDE18R8MB',  output_format='pandas')

stock_tables = ['netflix_stock', 'disney_stock', 'amazon_stock', 'google_stock']
stock = ['NFLX', 'DIS', 'AMZN', 'GOOGL']
i = 0
for table in stock_tables:
    data, meta_data = ts.get_daily(stock[i])
    data.reset_index(level=0, inplace=True)
    data = data[data.date > '2019-05-14'].reset_index(drop=True)
    data.columns = ['date', 'open', 'high', 'low', 'close', 'volume']
    data['Company'] = stock[i]
    data.to_sql(name = table, con = conn, index = False)
    i+=1